## NOTE - Use Python 3.6

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import pymongo
import dill
import os
from scipy import stats

from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from window_based_tagger_config import get_config
from FindFiles import find_files
from DirUtils import dir_exists
from results_procesor import ResultsProcessor, __MICRO_F1__
from Settings import Settings

In [2]:
FOLDER = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Predictions/"

In [3]:
def filter_by_str(filters, files, exclude=False):
    if type(filters) != list:
        filters = [filters]
    flt = files
    for fltr in filters:
        files = [f for f in files if (fltr in f) != exclude]
    return  files

In [4]:
all_files = os.listdir(FOLDER)
print(len(all_files))
files = list(all_files)
# files = filter_by_str("_TAGGING_", files, exclude=True)
files = filter_by_str("_FINAL_RUN_", files, exclude=True) # this is the sentence CREL parser (sentence level)
files = filter_by_str("_TAGGING_", files, exclude=True) # this is the sentence CREL parser (sentence level)
files = filter_by_str("_STACKED_", files, exclude=True) # this is the sentence CREL parser (sentence level)
files = filter_by_str("_PA_", files, exclude=True) # this is the sentence CREL parser (sentence level)
# files = filter_by_str("_CR_", files) 
files = filter_by_str("TEST_", files)
files = filter_by_str("_VD_", files)
print(len(files))
files = filter_by_str("2019", files, exclude=True)
print(len(files))

1202
163
15


In [5]:
sorted(files)

['SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_YS_.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS_.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_YS_.dill',
 'TEST_CB_STR_PCPTRN_RE-RANKER_VD_PREDS.dill',
 'TEST_CB_STR_PCPTRN_RE-RANKER_VD_YS_.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_PREDS__.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_YS_.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD_PREDS_.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD_YS_.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_PREDS__.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_YS_.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_YS__.dill',
 'TEST_SC_STR_PCPTRN_RE-RANKER_VD_2_PREDS__.dill',
 'TEST_SC_STR_PCPTRN_RE-RANKER_VD_2_YS_.dill']

In [6]:
sc_files = filter_by_str("_SC_", files)
cb_files = filter_by_str("_CB_", files)
len(cb_files), len(sc_files)

(8, 7)

In [7]:
cb_files

['TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD_YS_.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD_PREDS_.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_YS_.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_PREDS__.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__.dill',
 'TEST_CB_STR_PCPTRN_RE-RANKER_VD_YS_.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_YS_.dill',
 'TEST_CB_STR_PCPTRN_RE-RANKER_VD_PREDS.dill']

In [8]:
sc_files

['SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS_.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_YS_.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_YS_.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_YS__.dill',
 'TEST_SC_STR_PCPTRN_RE-RANKER_VD_2_YS_.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD_PREDS__.dill',
 'TEST_SC_STR_PCPTRN_RE-RANKER_VD_2_PREDS__.dill']

In [9]:
from collections import OrderedDict
filters = OrderedDict({
    "SENT_PARSER":  ["SENT_", "_PARSER_"],
    "ESSAY_PARSER": "SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2",
    "RE-RANKER":    "STR_PCPTRN_RE-RANKER"
})

In [10]:
from pprint import pprint

def load_predictions(input_files):
    algo2preds = dict()
    pred_files = filter_by_str("_PREDS", input_files)
    for algo_name, fltr in filters.items():
        f_files = filter_by_str(fltr, pred_files)
        assert len(f_files) == 1, (algo_name,f_files)
        fname = FOLDER + f_files[0]
        with open(fname, "rb+") as f:
            algo2preds[algo_name] = dill.load(f)
    return algo2preds

In [11]:
cb_preds = load_predictions(cb_files)
sc_preds = load_predictions(sc_files)

In [12]:
with open(FOLDER + "SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_YS_.dill", "rb+") as f:
    cb_ysbytag = dill.load(f)
    
with open(FOLDER + "SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_YS_.dill", "rb+") as f:
    sc_ysbytag = dill.load(f)

In [13]:
def get_all_preds(predsbytag, k_filter=None):
    if k_filter is None:
        k_filter = set(predsbytag.keys())
    all_p = []
    
    for k, vals in sorted(predsbytag.items(), key = lambda tpl: tpl[0]):
        if k not in k_filter:
            continue
        all_p.extend(vals)
    return all_p

## Get Predicted Tags from Labelled Essays

In [16]:
def print_comparison(ysbytag, algo2preds):
    algo2metrics = {}
    for algo, preds in algo2preds.items():
        mean_metrics = ResultsProcessor.compute_mean_metrics(ysbytag, preds)
        algo2metrics[algo] = mean_metrics[__MICRO_F1__]

    for algo_name_a, predsbytaga in algo2preds.items():
        f1_a = algo2metrics[algo_name_a]["f1_score"]
        rec_a = algo2metrics[algo_name_a]["recall"]
        prec_a = algo2metrics[algo_name_a]["precision"]
        print(f"{algo_name_a.ljust(20)} {f1_a:.4f} {rec_a:.4f} {prec_a:.4f}")


# CB

In [17]:
#perceptron and CRF differ somewhat from prior run
print_comparison(cb_ysbytag, cb_preds)

SENT_PARSER          0.7366 0.7642 0.7109
ESSAY_PARSER         0.7393 0.7258 0.7533
RE-RANKER            0.7500 0.7404 0.7598


# SC

In [18]:
# RNN only differs
print_comparison(sc_ysbytag, sc_preds)

SENT_PARSER          0.8271 0.7989 0.8573
ESSAY_PARSER         0.8210 0.8165 0.8256
RE-RANKER            0.8292 0.8055 0.8543
